### Change K, h_array according to the Table 1 in Dunnet's study

## B_delta0_array is about the P1(Overall power, 1-Beta in our study ) and regarding delta

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import root_scalar
import pandas as pd

def compute_integral(Nt, h, k, delta1_star, delta2_star, delta0_star, sigma, R, p1_star):
    def integrand(z):
        term1 = norm.cdf(z + (delta2_star * np.sqrt(Nt)) / (sigma * np.sqrt(R + k)))\
        ** (k - 1)
        inner_term = z - h * np.sqrt(1 + 1/R) + (delta1_star * np.sqrt(Nt)) /\
        (sigma * np.sqrt(R + k)) * (1 - delta0_star / delta1_star)
        term2 = norm.cdf(np.sqrt(R) * inner_term)
        return term1 * term2 * norm.pdf(z)

    result, _ = quad(integrand, -np.inf, np.inf)
    return result - p1_star

def find_Nt_given_combinations(k, delta1_star, delta2_star, sigma, R,B_delta0_array,h_array,p0_array):

    results = []

    for B, delta0 in B_delta0_array:
        for h, p0 in zip(h_array, p0_array):
            p1 = B

            f_low = compute_integral(1, h, k, delta1_star, delta2_star, delta0, sigma, R, p1)
            f_high = compute_integral(10000, h, k, delta1_star, delta2_star, delta0, sigma, R, p1)

            if f_low * f_high > 0:
                Nt = None
            else:
                sol = root_scalar(
                    compute_integral,
                    args=(h, k, delta1_star, delta2_star, delta0, sigma, R, p1),
                    bracket=(1, 10000),
                    method='brentq'
                )
                Nt = sol.root if sol.converged else None

            results.append({
                'k': k,
                'delta0': delta0,
                'Nt': round(Nt, 4) if Nt is not None else None,
                'p1': round(p1, 4),
            })

    return pd.DataFrame(results)

# Parameters
# Overall power and gamma(delta0) values from this study's Table 5.1
k = 4
B_delta0_array = [(0.7, 0.1), (0.75, 0.1),(0.8,0.095),(0.85,0.090),(0.90,0.075)] 

# From Dunnett's study
h_array = [2.1603]
p0_array = [0.95]

delta1_star = 0.2
delta2_star = 0.2
sigma = 1
R = 1

# Run and show results
df = find_Nt_given_combinations(k, delta1_star, delta2_star, sigma, \
                                R,B_delta0_array,h_array,p0_array)
print(df)

   k  delta0         Nt    p1
0  4   0.100  7207.6169  0.70
1  4   0.100  8036.0329  0.75
2  4   0.095  8173.7243  0.80
3  4   0.090  8445.5408  0.85
4  4   0.075  7581.6591  0.90
